In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
# %pip install --upgrade openai
# !pip install groq


In [27]:
# Importing Necessary Libraries
import os
from groq import Groq
from groq import Client

# client = Groq(
#     api_key=os.environ.get("GROQ_API_KEY"),
# )
# # Initialize the Groq client
# client = Client()

In [12]:
# import os
# import openai
# from packaging import version

# required_version = version.parse("1.1.1")
# current_version = version.parse(openai.__version__)

# if current_version < required_version:
#     raise ValueError(f"Error: OpenAI version {openai.__version__}"
#                      " is less than the required version 1.1.1")
# else:
#     print("OpenAI version is compatible.")

# # -- Now we can get to it
# from openai import OpenAI
# client = OpenAI(api_key="sk-")  # should use env variable OPENAI_API_KEY

In [13]:
# import openai

# # Set your OpenAI API key
# openai.api_key = 'sk-proj-'

# # Example usage: Creating a chat completion
# # response = openai.chat.completions.create(
# #     model="gpt-3.5-turbo",
# #     messages=[
# #         {"role": "system", "content": "You are a helpful assistant."},
# #         {"role": "user", "content": "Hello, world!"}
# #     ]
# # )

# # Print the response
# print(response.choices[0].message['content'].strip())


In [14]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="mixtral-8x7b-32768",
)

print(chat_completion.choices[0].message.content)

Fast language models are important for a variety of reasons, including:

1. Real-time applications: Fast language models are essential for real-time applications such as speech recognition, machine translation, and chatbots. These applications require quick and accurate responses, and slow models can result in poor user experience.
2. Cost-effective: Fast language models are more cost-effective than slow models because they require less computational resources. This is especially important for large-scale applications that require processing large amounts of data.
3. Scalability: Fast language models are more scalable than slow models. They can handle larger datasets and more complex tasks without sacrificing performance.
4. Accessibility: Fast language models make natural language processing (NLP) technologies accessible to a wider audience, including developers and researchers who may not have access to powerful computational resources.
5. Research and development: Fast language mode

In [15]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [16]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='mixtral-8x7b-32768',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
rag(query)

'To run Apache Kafka, you need to have Kafka installed and running in your environment. However, the FAQ provided seems to focus more on running Kafka-related applications such as producers and consumers.\n\nBased on the context, here are the steps to run a Python Kafka producer:\n\n1. Create a virtual environment and install the necessary packages by running the following commands:\n   ```\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n   Note: Replace `python` with `python3` if your system uses Python 3 by default.\n\n2. Install `kafka-python-ng` by running the following command:\n   ```\n   pip install kafka-python-ng\n   ```\n\n3. Run your Python Kafka producer script in the virtual environment.\n\nRegarding the other questions in the context, they are about resolving specific issues related to running Kafka applications, such as:\n\n- How to run a Java Kafka producer/consumer in the terminal.\n- How to install dependencies to run

In [20]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. According to the course guidelines, even if you don't register, you can still submit the homeworks, although there will be deadlines for turning in the final projects. It is recommended to not leave everything for the last minute.\n\nAdditionally, all the materials will be kept after the course finishes, so you can follow the course at your own pace after it finishes. This means that you can catch up with any missed content or lessons.\n\nBefore the course starts, you can start by installing and setting up all the dependencies and requirements, and looking over the prerequisites and syllabus to see if you are comfortable with these subjects.\n\nFinally, support is available even if you take the course in the self-paced mode. You can ask questions in the Slack channel, but it is recommended to search the channel and the FAQ document before asking, as most questions are already answered there."

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [22]:
from elasticsearch import Elasticsearch

In [23]:
es_client = Elasticsearch('http://localhost:9200') 

In [25]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

# es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
query = 'I just disovered the course. Can I still join it?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag(query)

"Yes, you can still join the course! It's a self-paced course, so you can sign up and start learning at any time. There are no deadlines or start/end dates, so you can work through the material at your own pace. However, please note that while you can access the course content right away, some features like instructor feedback or participation in live sessions may not be available if they have already passed or if the course has already ended."

In [ ]:
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pytorch-lightning 1.6.5 has a non-standard dependency specifier torch>=1.8.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import tiktoken

# Replace 'mixtral-8x7b-32768' with your actual model identifier if different
model_identifier = 'mixtral-8x7b-32768'

# Get the tokenizer
tokenizer = tiktoken.get_encoding(model_identifier)

# Now you can use this tokenizer explicitly in your code
text = "Your text to tokenize"
tokens = tokenizer.encode(text)
print(tokens)


ValueError: Unknown encoding mixtral-8x7b-32768

In [ ]:
import tiktoken
encoding = tiktoken.encoding_for_model("mixtral-8x7b-32768")

KeyError: 'Could not automatically map mixtral-8x7b-32768 to a tokeniser. Please use `tiktok.get_encoding` to explicitly get the tokeniser you expect.'

In [26]:
!curl localhost:9200

{
  "name" : "26a347c5f2ee",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "l_Exj7NfScGsXBD4cApLdQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0  40985      0 --:--:-- --:--:-- --:--:-- 44916
